# Modelado de Series Temporales de Aparcamientos usando RNNs y optimización con Optuna
En este notebook se abordará un problema de predicción de series temporales utilizando datos de disponibilidad de parkings.

Flujo de trabajo:
1. Agrupación de los datos por id de aparcamiento (idAparcamiento), para tratar cada parking como una serie temporal independiente.
2. División del conjunto de datos en tres subconjuntos: entrenamiento (train), validación (val) y prueba (test).
3. Entrenamiento de modelos de redes neuronales recurrentes simples (vanilla):
 - Vanilla RNN
 - Vanilla GRU
 - Vanilla LSTM
4. Ajuste de hiperparámetros utilizando Optuna para encontrar la configuración óptima en cada tipo de modelo.
5. Comparación de los resultados de rendimiento entre los distintos modelos utilizando métricas apropiadas.

El objetivo final es evaluar qué arquitectura ofrece mejores resultados para este tipo de datos y tarea de predicción.


In [102]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torch.optim as optim

import torchmetrics
from torchmetrics import MetricCollection
from torchmetrics.regression import MeanAbsoluteError, MeanSquaredError, MeanAbsolutePercentageError, R2Score

import optuna


## 1. Cargar los datos

In [85]:
df = pd.read_csv("../data/processed/data_processed.csv")

#convertir a indice
df.set_index("timestamp", inplace= True)
df.index = pd.to_datetime(df.index)

df

,idAparcamiento,PlazasTotales,PlazasDisponibles,PorcPlazasDisponibles,year,month,day,weekday
timestamp,,,,,,,,
2023-02-03 10:00:00,6,372.0,60.0,16.129032,2023,2,3,4
2023-02-03 11:00:00,6,372.0,48.0,12.903226,2023,2,3,4
2023-02-03 12:00:00,6,372.0,66.0,17.741935,2023,2,3,4
2023-02-03 13:00:00,6,372.0,119.0,31.989247,2023,2,3,4
2023-02-03 14:00:00,6,372.0,155.0,41.666667,2023,2,3,4
...,...,...,...,...,...,...,...,...
2025-03-05 03:00:00,78,464.0,355.0,76.508621,2025,3,5,2
2025-03-05 04:00:00,78,464.0,355.0,76.508621,2025,3,5,2
2025-03-05 05:00:00,78,464.0,356.0,76.724138,2025,3,5,2


## 2. División del conjunto de datos

Realizamos la división del conjunto de datos, agrupando por `idAparcamiento`. 
La idea es que el conjunto de **test sea común** para todos los aparcamientos, correspondiente al **último 10% del rango temporal total** del dataset. 

El resto de los datos disponibles para cada parking se dividen en:

- **Entrenamiento (train)**: el primer 85% de los datos previos al test.
- **Validación (val)**: el último 15% de los datos previos al test.

In [86]:
from datetime import timedelta

# 1. Calcular el rango temporal global
fecha_min_global = df.index.min()
fecha_max_global = df.index.max()
rango_total = fecha_max_global - fecha_min_global

# 2. Calcular el inicio del conjunto de test (último 10% del rango)
test_ratio = 0.10
test_duration = timedelta(seconds=rango_total.total_seconds() * test_ratio)
test_start = fecha_max_global - test_duration

# 3. Diccionarios para almacenar los splits
train_dict = {}
val_dict = {}
test_dict = {}

val_ratio = 0.1  # del conjunto anterior al test

# 4. División por parking
for parking_id, group in df.groupby("idAparcamiento"):
    group = group.sort_index()
    
    # Split basado en el corte global
    test_set = group[group.index >= test_start]
    remaining = group[group.index < test_start]

    # Dividir en train y val
    val_size = int(len(remaining) * val_ratio)
    val_set = remaining.iloc[-val_size:]
    train_set = remaining.iloc[:-val_size]
    
    # Guardar resultados
    train_dict[parking_id] = train_set
    val_dict[parking_id] = val_set
    test_dict[parking_id] = test_set




## 3. Definir `Dataset` de pytorch

In [87]:
class TimeSeriesDataset(Dataset):
    def __init__(self, data, input_window, output_window, feature_cols, target_col):
        """
        data: DataFrame que contiene los datos de la serie temporal
        input_window: nº de pasos de tiempo en la secuencia de entrada
        output_window: nº de pasos de tiempo a predecir
        feature_cols: lista de nombres de columnas que se usan como característcas
        target_col: nombre de la variable a predecir
        """
        self.data = data
        self.input_window = input_window
        self.output_window = output_window
        self.feature_cols = feature_cols
        self.target_cols = target_col

    def __len__(self):
        """
        Función que devuele el nº de datos del Dataset
        """
        return len(self.data) - self.input_window - self.output_window + 1 #

    def __getitem__(self, idx):
        """
        Función que devuelve un dato a partir de un índice
        """
        X = self.data[idx: idx + self.input_window][self.feature_cols].values
        Y = self.data[idx + self.input_window: idx + self.input_window + self.output_window][self.target_cols].values
        
        X_tensor = torch.tensor(X, dtype= torch.float32)
        Y_tensor = torch.tensor(Y, dtype= torch.float32)

        return X_tensor, Y_tensor

In [88]:
feature_cols = ['PorcPlazasDisponibles']  
target_col = 'PorcPlazasDisponibles'     
input_window = 24         # Número de pasos de tiempo en la secuencia de entrada
output_window = 1         # Número de pasos de tiempo a predecir

Dado que tenemos una serie temporal por parking, construimos un diccionario, donde las claves son los identificadores de parkings y los valores son Daatasets de tipo `torch.utils.data.Dataset`.

In [89]:
train_datasets = {
    pid: TimeSeriesDataset(df, input_window, output_window, feature_cols, target_col=target_col)
    for pid, df in train_dict.items()
}
val_datasets = {
    pid: TimeSeriesDataset(df, input_window, output_window, feature_cols, target_col=target_col)
    for pid, df in val_dict.items()
}
test_datasets = {
    pid: TimeSeriesDataset(df, input_window, output_window, feature_cols, target_col=target_col)
    for pid, df in test_dict.items()
}

In [90]:
for pid, df in train_datasets.items():
    print("id parking: ", pid)
    print("longitud: ", len(df))

id parking:  6
longitud:  13908
id parking:  7
longitud:  13908
id parking:  8
longitud:  22507
id parking:  13
longitud:  13908
id parking:  34
longitud:  9310
id parking:  75
longitud:  22505
id parking:  77
longitud:  10735
id parking:  78
longitud:  7454


## 4. Crear `DataLoaders` a partir de `Dataset`

Para crear los `DataLoader`, seguimos el mismo criterio, es decir, crear un diccionario de DataLoaders, en el que cada iteración nos devuelve un batch de datos para cada parking.
- Cada batch de datos debe tener dimensión: `(batch_size, window_size, n_features)`

In [91]:
train_dataloaders = {
    pid: DataLoader(ts_dataset, batch_size = 32, shuffle = True) #(batch_size, window_size, n_features)
    for pid, ts_dataset in train_datasets.items()
}

val_dataloaders = {
    pid: DataLoader(ts_dataset, batch_size = 32, shuffle = True) #(batch_size, window_size, n_features)
    for pid, ts_dataset in val_datasets.items()
}

test_dataloaders = {
    pid: DataLoader(ts_dataset, batch_size = 64) #(batch_size, window_size, n_features)
    for pid, ts_dataset in test_datasets.items()
}

In [92]:
for pid, dloader in train_dataloaders.items():
    print("id parking", pid)
    for batch in dloader:
        print("Dimensión del primer batch de datos:", batch[0].shape)
        print("Dimensión del primer batch de etiquetas: ", batch[1].shape)
        break

    print("\n")

id parking 6
Dimensión del primer batch de datos: torch.Size([32, 24, 1])
Dimensión del primer batch de etiquetas:  torch.Size([32, 1])


id parking 7
Dimensión del primer batch de datos: torch.Size([32, 24, 1])
Dimensión del primer batch de etiquetas:  torch.Size([32, 1])


id parking 8
Dimensión del primer batch de datos: torch.Size([32, 24, 1])
Dimensión del primer batch de etiquetas:  torch.Size([32, 1])


id parking 13
Dimensión del primer batch de datos: torch.Size([32, 24, 1])
Dimensión del primer batch de etiquetas:  torch.Size([32, 1])


id parking 34
Dimensión del primer batch de datos: torch.Size([32, 24, 1])
Dimensión del primer batch de etiquetas:  torch.Size([32, 1])


id parking 75
Dimensión del primer batch de datos: torch.Size([32, 24, 1])
Dimensión del primer batch de etiquetas:  torch.Size([32, 1])


id parking 77
Dimensión del primer batch de datos: torch.Size([32, 24, 1])
Dimensión del primer batch de etiquetas:  torch.Size([32, 1])


id parking 78
Dimensión del pr

## 5. Definir Modelos (`RNN`,`GRU`,`LSTM`)


In [121]:
class VanillaRNN(nn.Module):
    """
    Clase que implementa una RNN vanilla: 1 capa, 1 neurona por capa oculta
    """
    def __init__(self, input_size, hidden_size, output_size, num_layers = 1):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

        self.rnn = nn.RNN(input_size = self.input_size,
                          hidden_size = self.hidden_size,
                          num_layers = self.num_layers,
                          batch_first = True)
        self.fc = nn.Linear(self.hidden_size, self.output_size)
    
    def forward(self, x, h0 = None):
        """ 
        
        """
        batch_size_ = x.size(0)
        #si no se pasa el primer estado oculto, lo inicializamos con 0
        if h0 is None:
            h0 = torch.zeros(self.num_layers, batch_size_, self.hidden_size).to(x.device)
        
        #all_hidden_states (output) --> todos los estados ocultos (batch_size, seq_length, hidden_size)
        #last_hidden_state (h_n) --> ultimo estado oculto (num_layers, batch_size, hidden_size)
        all_hidden_states, last_hidden_state = self.rnn(x, h0)

        last_hidde_state_of_last_layer = last_hidden_state[-1]
        
        #si no pasamos por una capa densa, el modelo no hace la prediccion
        pred = self.fc(last_hidde_state_of_last_layer)  # (batch_size, output_size)

        return pred

Comprobamos que no hay inconsistencias

In [94]:
model = VanillaRNN(input_size=1, hidden_size=64, output_size=1, num_layers=1)
x = torch.randn(32, 24, 1)  # (batch_size, sequence_length, num_features)
y  = model(x)
print(y.shape)  # → (32, 1)


torch.Size([32, 1])


In [96]:
import torch
import torch.nn as nn

class VanillaLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(
            input_size=self.input_size,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
            batch_first=True
        )
        
        self.fc = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, x, h0=None, c0=None):
        batch_size_ = x.size(0)
        #si no se pasa el primer h0 o c0, lo inicializamos con 0
        if h0 is None:
            h0 = torch.zeros(self.num_layers, batch_size_, self.hidden_size).to(x.device)
        if c0 is None:
            c0 = torch.zeros(self.num_layers, batch_size_, self.hidden_size).to(x.device)

        all_hidden_states, (last_hidden_state, last_cell_memory) = self.lstm(x, (h0, c0))

        # Usar el último estado oculto de la última capa
        pred = self.fc(last_hidden_state[-1])  # Shape: (batch_size, output_size)

        return pred


Comprobamos que no hay inconsistencias

In [97]:
model = VanillaLSTM(input_size=1, hidden_size=64, output_size=1, num_layers=1)
x = torch.randn(32, 24, 1)  # (batch_size, sequence_length, num_features)
y  = model(x)
print(y.shape)  #(32, 1)

torch.Size([32, 1])


In [98]:
class VanillaGRU(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers = 1):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

        self.gru = nn.GRU(input_size = self.input_size,
                          hidden_size = self.hidden_size,
                          num_layers = self.num_layers,
                          batch_first= True)
        
        self.fc = nn.Linear(self.hidden_size, self.output_size)
    
    def forward(self, x, h0=None):
        
        batch_size_ = x.size(0)
        
        ##si no se pasa el primer h0, lo inicializamos con 0
        if h0 is None:
            h0 = torch.zeros(self.num_layers, batch_size_, self.hidden_size).to(x.device)
        
        #all_hidden_states (output) --> todos los estados ocultos (batch_size, seq_length, hidden_size)
        #last_hidden_state (h_n) --> ultimo estado oculto (num_layers, batch_size, hidden_size)
        all_hidden_states, last_hidden_state = self.gru(x, h0)

        last_hidde_state_of_last_layer = last_hidden_state[-1]
        
        #si no pasamos por una capa densa, el modelo no hace la prediccion
        pred = self.fc(last_hidde_state_of_last_layer)  # (batch_size, output_size)

        return pred  



In [99]:
model = VanillaGRU(input_size=1, hidden_size=64, output_size=1, num_layers=1)
x = torch.randn(32, 24, 1)  # (batch_size, sequence_length, num_features)
y  = model(x)
print(y.shape)  #(32, 1)

torch.Size([32, 1])


## 6. Definir métricas

In [ ]:
metrics = MetricCollection({
    "MAE": MeanAbsoluteError(),
    "MSE": MeanSquaredError(),
    "RMSE": MeanSquaredError(squared = False),
    "MAPE": MeanAbsolutePercentageError()
})

## 7. Instanciar modelos, optimizador, función de coste y learning rate scheduler

In [119]:
for pid, dloader in train_dataloaders.items():
    print("id parking", pid)
    batch = next(iter(dloader))
    INPUT_SIZE = tuple(batch[0].shape)
    OUTPUT_SIZE = tuple(batch[1].shape)
    print("Dimensión del primer batch de datos:", INPUT_SIZE)
    print("Dimensión del primer batch de etiquetas:", OUTPUT_SIZE)
    break

id parking 6
Dimensión del primer batch de datos: (32, 24, 1)
Dimensión del primer batch de etiquetas: (32, 1)


In [122]:
vanilla_rnn = VanillaRNN(input_size = INPUT_SIZE, hidden_size = 32, output_size = OUTPUT_SIZE, num_layers=1)
vanilla_lstm = VanillaLSTM(input_size = INPUT_SIZE, hidden_size = 32, output_size = OUTPUT_SIZE, num_layers=1)
vanilla_gru = VanillaGRU(input_size = INPUT_SIZE, hidden_size = 32, output_size = OUTPUT_SIZE, num_layers=1)

TypeError: empty(): argument 'size' failed to unpack the object at pos 2 with error "type must be tuple of ints,but got tuple"

In [ ]:
criterion = nn.MSELoss()

In [ ]:
optimizer = optim.Adam()

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

## 8. Crear callbacks

## 9. Definir bucle de entrenamiento

## 10. Definir bucle de validación

## 11. Optimizació óptima de hiperparámetros con `Optuna`

## 12. Entrenamiento

## 13. Predicciones sobre el conjunto de test

## 14. Exportación de checkpoints y logs